# Homework 4

This Notebook builds on the DCOPF model introduced in [Notebook 6](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks) and incorporates some elements of Economic Dispatch introduced in [Notebook 4](https://github.com/east-winds/power-systems-optimization/tree/master/Notebooks).

First, load (or install if necessary) a set of packages you'll need for this assignment...

In [ ]:
using JuMP
using GLPK
using DataFrames
using CSV
using Plots; plotly();

## Question 1: Modifying IEEE-14

**A. Increased generation costs**

Copy the IEEE 14 bus system and DCOPF solver function from Notebook 6. In addition, add the following line to the return call of the function:
```julia
status = termination_status(DCOPF)
```
This tells you the solver termination status for the problem: e.g. was an optimal solution found, was the solution infeasible, was it unbounded, etc.

Make the following change to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh

Run the DCOPF and output generation, flows, and prices.

In [ ]:
datadir = joinpath("..","Notebooks","ieee_test_cases") 
gens = CSV.read(joinpath(datadir,"Gen14.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load14.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids 
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters
lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines2.f
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

# calculate simple susceptance, ignoring resistance as earlier 
lines.b = 1 ./ lines.reactance

# keep only a single time period
loads = loads[:,["connnode","interval-1_load"]]
rename!(loads,"interval-1_load" => "demand");

Regarding the above results, answer the following:

- How has generation changed compared to the default system?
- What explains the new prices?


**B. Constrained line**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW

Run the DCOPF and output generation, flows, and prices.

Regarding the above results, answer the following:

- Which node has the highest price and why?
- What is the difference in prices across $l_{23}$, also known as the congestion rent? How do you interpret this value (what is it's practical meaning?)

**C. Demand increase**

Make the following changes to the system:

- Increase the variable cost of Generator 1 to \$30 / MWh
- Reduce flow limit on the line connecting 2 and 3 ($l_{23}$) to 70 MW
- Increase demands everywhere by 5\%.

Calculate the total available generating capacity:

Calculate the new total demand:

Run the DCOPF and show prices.

**What is happening in this system?** 

## Question 2: Linear losses

Up until now, we have ignored transmission losses. A quadratic approximation of losses is given by:

\begin{align}
LOSS_{ij} &\approx \frac{G_{ij}}{BaseMVA} (\theta_i-\theta_j)^2 \\
 & \approx \frac{1}{BaseMVA} \frac{R_{ij}}{R_{ij}^2+X_{ij}^2}(\theta_i-\theta_j)^2
\end{align}


where $G$ is the line's conductance, $R$ is the line's resistance and $X$ is the line's reactance. See the `lines` data frame for these parameters.

For our purposes, we will approximate this quadratic via:


$$
LOSS_{ij} \geq \frac{R_{ij}}{BaseMVA} \times (MaxFlow_{ij})^2 
\left(\frac{|FLOW_{ij}|}{MaxFlow_{ij}} - 0.165 \right)
$$

where $MaxFlow_{ij}=200 MW$ in this problem. Note the greater than equal sign, as we do not want to have negative losses.

This approximation is based on Fitiwi et al. (2016), "Finding a representative network losses model for large-scale transmission expansion planning with renewable energy sources," *Energy* 101: 343-358, https://doi.org/10.1016/j.energy.2016.02.015. 

Note that this is a linear approximation of transmission losses, which are actually a quadratic function of power flows. Fitiwi et al. 2016 and other papers describe piece-wise or segment-wise linear approximations of the quadratic function which provide a tighter lower bound approximation of losses, but we'll use a single linear term for this assignment. 

See Jenkins & Sepulveda et al. 2017, "Enhanced decision support for a changing electricity landscape: the GenX configurable electricity resource capacity expansion model", MIT Energy Initiative Working Paper 2017-10 http://bit.ly/GenXModel Section 5.8, for an example of a linear segment-wise approximation of quadratic transmission losses. 


**A. Code linear losses**

Reload the original data from Notebook 6 and copy the IEEE 14 bus system and DCOPF solver function from Notebook 6 into a new function `dcopf_ieee_lossy`.

Make the following changes:
- Increase the variable cost of Generator 1 to \$30 / MWh
- Change all transmission line capacities to 200 MW

Implement losses into the supply/demand balance equations. A standard way to implement absolute values in linear programming is by introducing two non-negative auxiliary variables $x^+$, $x^-$ $\geq 0$:

$$
x = x^+ - x^-
$$

and the absolute value can be represented as:

$$
|x| = x^+ + x^-
$$

(You should satisfy yourself that this equality holds.)

It makes the formulation easier if losses are added to the supply/demand balance constraint in each node by splitting losses in half between the receiving and sending end.

Indicate which equations and variables you have added and explain your steps using inline code comments (e.g. `# Comment`).

Run the lossy DCOPF and output generation, flows, losses, and prices.

**B. Interpret results**

Run the same parameters in the lossless OPF from problem 1. How do prices and flows change? What is the largest magnitude difference in prices between the solution with losses and the lossless OPF solution?

## Question 3 - Security contingencies

Power system operators need to ensure that power is delivered reliably even in the event of unexpected outages (**contingencies**). One common contigency that must be planned for is the loss of a transmission line. The security-constrained OPF (SCOPF) run by operators solves for an optimal dispatch that is simultaneously robust (i.e., feasible) to each of the lines failing individually. This is what is known as **N-1 security**, because we assume that at most one component fails in any given scenario.

In this problem, we will not code a full SCOPF, but rather investigate what happens to the feasibility of our problem when we remove transmission lines.

**A. Setup data**

The following code loads the original dataset (with one row per line) and includes a function `format_lines` that converts this to a format that our solver function can use (duplicating rows for both directions, adding susceptance, etc.).

In [ ]:
lines = CSV.read(joinpath(datadir,"Tran14.csv"), DataFrame);
rename!(lines,lowercase.(names(lines)))

function format_lines(lines)
    # create line ids 
    lines.id = 1:nrow(lines);
    # add set of rows for reverse direction with same parameters
    lines2 = copy(lines)
    lines2.f = lines2.fromnode
    lines2.fromnode = lines.tonode
    lines2.tonode = lines2.f
    lines2 = lines2[:,names(lines)]
    append!(lines,lines2)

    # calculate simple susceptance, ignoring resistance as earlier 
    lines.b = 1 ./ lines.reactance
    return(lines)
end

Next:

1. Set the capacity of all lines in the system at 100 MW, except for the line $l_{12}$, which you should set to 200 MW.

2. Create a load dataframe `loads_sens` that increases demands everywhere by 10\%

**B. Loop over line contingencies**

Create a dataframe `status` with the `fromnode` and `tonode` columns of `lines`.

Create a [for loop](https://docs.julialang.org/en/v1/manual/control-flow/#man-loops) that iterates over each line and:
- sets the reactance to be a very high value, 1e9 (i.e., no power will be transmitted)
- creates a version of the dataframe that our solver function can use via `format_lines`
- runs DCOPF
- stores the solution status in a `opf` column in the corresponding row of the `status` dataframe

Show the `status` results.

**3. Interpret results**

Are all of the cases feasible? If not, how many are infeasible? 

Pick two cases where the solution gives a different status. (For our purposes, dual infeasible and primal infeasible are the same.) What is happening here?

Given this, do you conclude that the system with the assumed transmission line ratings is secure as-is, or do we need to add more redundancy to the system?